<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.9

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

#class instantiations
ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

def BigDawg(X, y, params, task, ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  '''facilitates the Data Collection Process'''
  dl = DeepLearning()
  wr = Wrappers()
  if task == 'NLP':
    #if regression==False:
    X2, y2, _ = wr.TCP(X, y)
    #else:
      #X2, y2, _ = wr.TRP(X, y)
    func = dl.RNN 
  if task == 'CV':
    X2, _, y2, _ = dl.ModelReadyPixles(X, y, target_size=(ts, ts))
    func = dl.CNN
  if task == 'TC':
    X2 = np.array(X)
    y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularClassification
  if task == 'TR':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularRegression
  try:
    df = dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  except:
    df = dl.CollectPerformance(params, func, X2, y2.reshape(-1), epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  size = []
  n_features = []
  batch = []
  depth = []
  dominances = []
  for i in range(len(df)):
    size.append(X2.shape[0])
    n_features.append(X2.shape[1])
    dominances.append(max(pd.DataFrame(y2).value_counts())/len(y2))
    if task == 'CV':
      depth.append(X2.shape[3])
  df['len_dataset'] = size
  df['n_features'] = n_features
  if regression == False:
    df['dominant_class'] = dominances
  if task == 'CV':
    df['thickness'] = depth
  return df

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = load_breast_cancer()['target']
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

In [21]:
X = articles
y = labels

df = Wikipedia
data_str = 'Text'
output = len(np.unique(y))
embedding = dh.GetVocab(df, data_str)
rnn_grid = {'output_dim': [output], 'embedding': [embedding], 'nodes': [32, 40, 80, 100, 120], 'activation': ['tanh'], 'regularizer': ['L1', None, 'L2'], 
            'stacking': [False, True], 'dropout': [False, True], 'optimizer': ['rmsprop'], 'method': ['GRU', 'LSTM'], 'bidirectional': [True]}

#output = len(y)
#cnn_grid = {'output_dim': [output], 'base_filters':[32], 'kernel_size': [2], 'activation': ['relu'], 'nblocks': [3], 
            #'pool':[2], 'dropout': [False], 'closer': [False], 'optimizer': ['adam'], 'metrics': ['accuracy']}

#output = len(np.unique(y))
#tc_grid = {'output_dim': [output], 'nodes': [64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 
           #'dropout': [False], 'nlayers': [4], 'closer': [False], 'loss_func': ['binary_crossentropy'], 'optimizer': ['adam']}

#tr_grid = {'nodes': [32, 64, 128], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 'dropout': [False], 
           #'nlayers': [4], 'closer': [False], 'loss_func': ['MSE', 'MAE'], 'optimizer': ['rmsprop'], 'y_col': ['string']}

df10 = BigDawg(X, y, rnn_grid, 'NLP')
df10.shape

preforming 120 total fits ...
Epoch 1/50
preforming 120 total fits ...
Epoch 1/50
9/9 [==============================] - 5s 205ms/step - loss: 0.6956 - accuracy: 0.4765 - val_loss: 0.6932 - val_accuracy: 0.5417
Epoch 2/50
9/9 [==============================] - 1s 124ms/step - loss: 0.6816 - accuracy: 0.6701 - val_loss: 0.6930 - val_accuracy: 0.5278
Epoch 3/50
9/9 [==============================] - 1s 128ms/step - loss: 0.6603 - accuracy: 0.7693 - val_loss: 0.6912 - val_accuracy: 0.5556
Epoch 4/50
9/9 [==============================] - 1s 124ms/step - loss: 0.6047 - accuracy: 0.9060 - val_loss: 0.6898 - val_accuracy: 0.5972
Epoch 5/50
9/9 [==============================] - 1s 124ms/step - loss: 0.5020 - accuracy: 0.9599 - val_loss: 0.7094 - val_accuracy: 0.5833
Epoch 6/50
9/9 [==============================] - 1s 123ms/step - loss: 0.3049 - accuracy: 0.9596 - val_loss: 0.8225 - val_accuracy: 0.5694
Epoch 7/50
9/9 [==============================] - 1s 123ms/step - loss: 0.1676 - accuracy

(120, 19)

In [22]:
df0 = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/NLP4.csv').drop(['Unnamed: 0'], axis='columns')

results = pd.concat([df0, df6, df7, df8, df9, df10])

In [23]:
results

,output_dim,embedding,nodes,activation,regularizer,stacking,dropout,optimizer,bidirectional,epochs,batch_size,method,accuracy,loss,val_accuracy,val_loss,len_dataset,n_features,dominant_class
0,3,12300,64,tanh,NaN,False,False,adam,False,7,32,GRU,0.844152,0.307092,0.565996,1.194260,8936,17,0.602954
1,3,12300,64,tanh,NaN,True,False,adam,False,7,32,GRU,0.840095,0.310448,0.577740,1.137914,8936,17,0.602954
2,3,12300,64,tanh,L1,False,False,adam,False,7,32,GRU,0.840795,0.310173,0.576622,1.128422,8936,17,0.602954
3,3,12300,64,tanh,L1,True,False,adam,False,6,32,GRU,0.837577,0.334755,0.576622,0.993850,8936,17,0.602954
4,2,45589,64,tanh,NaN,False,False,adam,False,6,32,GRU,0.625000,1.135028,0.000000,2.678114,360,1724,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2,45589,120,tanh,L2,False,True,rmsprop,True,7,32,LSTM,0.923611,0.449481,0.500000,0.969257,360,1724,0.500000
116,2,45589,120,tanh,L2,True,False,rmsprop,True,6,32,GRU,0.965278,0.190953,0.402778,0.925385,360,1724,0.500000
117,2,45589,120,tanh,L2,True,False,rmsprop,True,6,32,LSTM,0.913194,0.485912,0.472222,0.842673,360,1724,0.500000
118,2,45589,120,tanh,L2,True,True,rmsprop,True,7,32,GRU,0.986111,0.156143,0.472222,0.958312,360,1724,0.500000


In [24]:
results.to_csv('drive/MyDrive/NLP5.csv')

# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']